In [ ]:
import os
import numpy as np
from sklearn.metrics import balanced_accuracy_score
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import backend as K

In [ ]:
train_images_dir = 'train_images_dir'
train_masks_dir  = 'train_mask_dir'
test_dir         = 'test_dir'
img_size         = (128, 128)
batch_size       = 8
epochs           = 100

In [9]:
def load_dataset(images_dir, masks_dir, img_size):
    X, Y = [], []
    for fname in sorted(os.listdir(images_dir)):
        if not fname.lower().endswith('.jpg'):
            continue
        # Imagen en gris, normalizada a [0,1]
        img = load_img(os.path.join(images_dir, fname),
                       target_size=img_size,
                       color_mode='grayscale')
        img = img_to_array(img) / 255.0  # dtype float32
        # Máscara, umbral y cast a float32
        mask = load_img(os.path.join(masks_dir, fname.replace('.jpg','.png')),
                        target_size=img_size,
                        color_mode='grayscale')
        mask = img_to_array(mask) / 255.0
        mask = (mask > 0.5).astype('float32')    # <-- aquí

        X.append(img)
        Y.append(mask)
    return np.stack(X), np.stack(Y)

In [10]:
def load_test_dataset(test_dir, img_size):
    X, Y = [], []
    for fname in sorted(os.listdir(test_dir)):
        if not fname.lower().endswith('.jpg'):
            continue
        img = load_img(os.path.join(test_dir, fname),
                       target_size=img_size,
                       color_mode='grayscale')
        img = img_to_array(img) / 255.0
        mask = load_img(os.path.join(test_dir, fname.replace('.jpg','.png')),
                        target_size=img_size,
                        color_mode='grayscale')
        mask = img_to_array(mask) / 255.0
        mask = (mask > 0.5).astype(np.uint8)
        X.append(img)
        Y.append(mask)
    return np.stack(X), np.stack(Y)

In [11]:
def unet_model(input_shape):
    inputs = Input(input_shape)

    # Encoder
    c1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
    c1 = Conv2D(64, 3, activation='relu', padding='same')(c1)
    p1 = MaxPooling2D()(c1)

    c2 = Conv2D(128, 3, activation='relu', padding='same')(p1)
    c2 = Conv2D(128, 3, activation='relu', padding='same')(c2)
    p2 = MaxPooling2D()(c2)

    c3 = Conv2D(256, 3, activation='relu', padding='same')(p2)
    c3 = Conv2D(256, 3, activation='relu', padding='same')(c3)
    p3 = MaxPooling2D()(c3)

    # Bottleneck
    c4 = Conv2D(512, 3, activation='relu', padding='same')(p3)
    c4 = Conv2D(512, 3, activation='relu', padding='same')(c4)

    # Decoder
    u5 = Conv2DTranspose(256, 2, strides=2, padding='same')(c4)
    u5 = concatenate([u5, c3])
    c5 = Conv2D(256, 3, activation='relu', padding='same')(u5)
    c5 = Conv2D(256, 3, activation='relu', padding='same')(c5)

    u6 = Conv2DTranspose(128, 2, strides=2, padding='same')(c5)
    u6 = concatenate([u6, c2])
    c6 = Conv2D(128, 3, activation='relu', padding='same')(u6)
    c6 = Conv2D(128, 3, activation='relu', padding='same')(c6)

    u7 = Conv2DTranspose(64, 2, strides=2, padding='same')(c6)
    u7 = concatenate([u7, c1])
    c7 = Conv2D(64, 3, activation='relu', padding='same')(u7)
    c7 = Conv2D(64, 3, activation='relu', padding='same')(c7)

    outputs = Conv2D(1, 1, activation='sigmoid')(c7)
    return Model(inputs, outputs)

In [12]:
X_train, y_train = load_dataset(train_images_dir, train_masks_dir, img_size)
X_test,  y_test  = load_test_dataset(test_dir, img_size)

In [15]:
class BalancedAccuracy(tf.keras.metrics.Metric):
    def __init__(self, name='balanced_accuracy', **kwargs):
        super().__init__(name=name, **kwargs)
        # fuerza dtype float32
        self.tp = self.add_weight(name='tp', shape=(), dtype=tf.float32, initializer='zeros')
        self.tn = self.add_weight(name='tn', shape=(), dtype=tf.float32, initializer='zeros')
        self.fp = self.add_weight(name='fp', shape=(), dtype=tf.float32, initializer='zeros')
        self.fn = self.add_weight(name='fn', shape=(), dtype=tf.float32, initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_pred = tf.cast(y_pred > 0.5, tf.float32)
        y_true = tf.cast(y_true, tf.float32)

        # suma los TP, TN, FP, FN convirtiendo antes a float
        self.tp.assign_add(tf.reduce_sum(tf.cast(
            tf.logical_and(tf.equal(y_true, 1), tf.equal(y_pred, 1)), tf.float32)))
        self.tn.assign_add(tf.reduce_sum(tf.cast(
            tf.logical_and(tf.equal(y_true, 0), tf.equal(y_pred, 0)), tf.float32)))
        self.fp.assign_add(tf.reduce_sum(tf.cast(
            tf.logical_and(tf.equal(y_true, 0), tf.equal(y_pred, 1)), tf.float32)))
        self.fn.assign_add(tf.reduce_sum(tf.cast(
            tf.logical_and(tf.equal(y_true, 1), tf.equal(y_pred, 0)), tf.float32)))

    def result(self):
        sens = self.tp / (self.tp + self.fn + 1e-7)
        spec = self.tn / (self.tn + self.fp + 1e-7)
        return (sens + spec) / 2

    def reset_states(self):
        for v in self.variables:
            v.assign(0.0)


In [29]:
def dice_coef(y_true, y_pred, smooth=1e-6):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)


In [30]:
model = unet_model((128, 128, 1))
model.compile(optimizer='adam', loss=dice_loss, metrics=['accuracy', BalancedAccuracy()])

In [31]:
model.fit(
    X_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.1
)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.0257 - balanced_accuracy: 0.5040 - loss: 0.9657 - val_accuracy: 0.0198 - val_balanced_accuracy: 0.5088 - val_loss: 0.9957
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 249ms/step - accuracy: 0.0345 - balanced_accuracy: 0.5068 - loss: 0.9655 - val_accuracy: 0.0191 - val_balanced_accuracy: 0.5085 - val_loss: 0.9957
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step - accuracy: 0.0391 - balanced_accuracy: 0.5109 - loss: 0.9654 - val_accuracy: 0.5950 - val_balanced_accuracy: 0.7828 - val_loss: 0.9957
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step - accuracy: 0.6214 - balanced_accuracy: 0.7568 - loss: 0.9651 - val_accuracy: 0.6816 - val_balanced_accuracy: 0.7835 - val_loss: 0.9956
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step - accuracy: 0.7144 - balanced_accuracy: 0.7637 - loss: 0.9647 - val_accuracy: 0.7980 - val_balanced_accuracy: 0.7562 - val_loss: 0.9955
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 254ms/step - accuracy: 0.817

In [ ]:
y_pred        = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(np.uint8)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step


In [ ]:
plt.imsave("pred_unet.jpg", y_pred_binary.squeeze(), cmap='gray')